In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Read Data

In [ ]:
df=pd.read_csv("../input/donorsprediction/Raw_Data_for_train_test.csv")

In [ ]:
df

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.describe()

# Find missing values

In [ ]:
df.isnull().sum()

In [ ]:
df.columns[df.isnull().any()]

# Missing values

In [ ]:
df.isnull().sum()

In [ ]:
df['TARGET_D']=df['TARGET_D'].fillna(df['TARGET_D'].mean())

# create a new column(DONATION_TYPE)

In [ ]:
# create a list of our conditions
conditions = [
    (df['TARGET_D'] >= 50),
    (df['TARGET_D'] >= 20) & (df['TARGET_D'] < 50),
    (df['TARGET_D'] >= 13) & (df['TARGET_D'] < 20),
    (df['TARGET_D'] >= 10) & (df['TARGET_D'] < 13),
    (df['TARGET_D'] < 10)
    ]

# create a list of the values we want to assign for each condition
values = ['A', 'B', 'C', 'D','E']

# create a new column and use np.select to assign values to it using our lists as arguments
df['DONATION_TYPE'] = np.select(conditions, values)

# display updated DataFrame
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
#target d contains a lot of null values ,so delete this column
df=df.drop(['TARGET_D'],axis=1)

In [ ]:
df=df.drop(['PUBLISHED_PHONE'],axis=1)

In [ ]:
df['DONOR_AGE']=df['DONOR_AGE'].fillna(df['DONOR_AGE'].mean())

In [ ]:
df['DONOR_AGE']=df['DONOR_AGE'].astype('int64')

In [ ]:
df['INCOME_GROUP']=df['INCOME_GROUP'].fillna(df['INCOME_GROUP'].mode()[0])

In [ ]:
df['INCOME_GROUP']=df['INCOME_GROUP'].astype('int64')

In [ ]:
df['WEALTH_RATING']=df['WEALTH_RATING'].fillna(df['WEALTH_RATING'].mode()[0])

In [ ]:
df['WEALTH_RATING']=df['WEALTH_RATING'].astype('int64')

In [ ]:
df=df.dropna()

In [ ]:
df.columns[df.isnull().any()]

In [ ]:
df

In [ ]:
df['SES'].value_counts()

In [ ]:
df['SES']=df['SES'].str.replace('?','2')
df['SES']=df['SES'].astype('int64')

In [ ]:
df['URBANICITY'].value_counts()

In [ ]:
df['URBANICITY']=df['URBANICITY'].str.replace('?','S')

In [ ]:
df['CLUSTER_CODE'].value_counts()

In [ ]:
df['CLUSTER_CODE']=df['CLUSTER_CODE'].str.replace('.','40')
df['CLUSTER_CODE']=df['CLUSTER_CODE'].astype('int64')

In [ ]:
# Get list of categorical variables
s = (df.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

# make label encoder to categorical coulmns

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
for i in list(df.columns):
    if df[i].dtype=='object':
        df[i]=le.fit_transform(df[i])

In [ ]:
df.dtypes

In [ ]:
df

# Apply K Means

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
from sklearn.cluster import KMeans

In [ ]:
# Using the elbow method to find the optimal number of clusters
from sklearn.cluster import KMeans
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
    kmeans.fit(X)
    # inertia method returns wcss for that model
    wcss.append(kmeans.inertia_)

In [ ]:
plt.figure(figsize=(10,5))
sns.lineplot(range(1, 11), wcss,marker='o',color='red')
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
# Fitting K-Means to the dataset
kmeans = KMeans(n_clusters = 2, init = 'k-means++', random_state = 42)
y_kmeans = kmeans.fit_predict(X)

# Visulization

In [ ]:
# Generate scatter plot for training data
colors = list(map(lambda x: '#3b4cc0' if x == 1 else '#b40426', y_kmeans))
plt.scatter(X[:,0], X[:,1], c=colors, marker="o", picker=True)
plt.title('Two clusters of data')
plt.xlabel('Temperature yesterday')
plt.ylabel('Temperature today')
plt.show()

In [ ]:
# Fitting K-Means to the dataset
kmeans = KMeans(n_clusters = 3, init = 'k-means++', random_state = 42)
y_kmeans = kmeans.fit_predict(X)

In [ ]:
# Visualising the clusters
plt.figure(figsize=(15,7))
sns.scatterplot(X[y_kmeans == 0, 0], X[y_kmeans == 0, 1], color = 'yellow', label = 'Cluster 1',s=50)
sns.scatterplot(X[y_kmeans == 1, 0], X[y_kmeans == 1, 1], color = 'blue', label = 'Cluster 2',s=50)
sns.scatterplot(X[y_kmeans == 2, 0], X[y_kmeans == 2, 1], color = 'green', label = 'Cluster 3',s=50)
sns.scatterplot(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], color = 'red', 
                label = 'Centroids',s=300,marker=',')
plt.grid(False)
plt.legend()
plt.show()

In [ ]:
#%% KMEANS wiht sklearn 
from sklearn.cluster import KMeans
wcss=[]
#find best k value
for k in range(1,15):
    kmeans=KMeans(n_clusters=k)
    kmeans.fit(df)
    wcss.append(kmeans.inertia_)
#elbow rule on plot    
plt.figure(figsize=(12,8))
plt.plot(range(1,15),wcss,"-o")
plt.title("wcss / number of cluster", fontsize=18)
plt.xlabel("number of k(cluster) values")
plt.xticks(range(1,15))
plt.grid(True)
plt.ylabel("wcss")
plt.tight_layout()
plt.show()

In [ ]:
#%%   from elbow plot we can choose 3 or 4 i'll go with 4 cluster.
#kmeans2=KMeans(n_clusters=2)
#kmeans3=KMeans(n_clusters=3)
kmeans=KMeans(n_clusters=4)
clusters=kmeans.fit_predict(df)
df["label"]=clusters

In [ ]:
#%% plot with cluster / center(centroid)
plt.figure(figsize=(20,8))
plt.scatter(df.CLUSTER_CODE[df.label==0],df.CONTROL_NUMBER[df.label==0],color="red",alpha= 0.8)
plt.scatter(df.CLUSTER_CODE[df.label==1],df.CONTROL_NUMBER[df.label==1],color="green",alpha= 0.8)
plt.scatter(df.CLUSTER_CODE[df.label==2],df.CONTROL_NUMBER[df.label==2],color="blue",alpha= 0.8)
plt.scatter(df.CLUSTER_CODE[df.label==3],df.CONTROL_NUMBER[df.label==3],color="black",alpha= 0.8)
plt.scatter(kmeans.cluster_centers_[:,0],kmeans.cluster_centers_[:,1],color="orange")# cluster center(centroid)
plt.title("Kmeans(k=4) Cluster")
plt.xlabel("CLUSTER_CODE")
plt.ylabel("CONTROL_NUMBER")
plt.legend(df,loc='upper right')
plt.show()

In [ ]:
#%% plot with cluster / center(centroid)
plt.figure(figsize=(20,8))
plt.scatter(df.DONOR_AGE [df.label==0],df.IN_HOUSE[df.label==0],color="red",alpha= 0.8)
plt.scatter(df.DONOR_AGE [df.label==1],df.IN_HOUSE[df.label==1],color="green",alpha= 0.8)
plt.scatter(df.DONOR_AGE [df.label==2],df.IN_HOUSE[df.label==2],color="blue",alpha= 0.8)
plt.scatter(df.DONOR_AGE [df.label==3],df.IN_HOUSE[df.label==3],color="black",alpha= 0.8)
plt.scatter(kmeans.cluster_centers_[:,0],kmeans.cluster_centers_[:,1],color="orange")# cluster center(centroid)
plt.title("Kmeans(k=4) Cluster")
plt.xlabel("DONOR_AGE ")
plt.ylabel("IN_HOUSE")
plt.legend(df,loc='upper right')
plt.show()

In [ ]:
#%%   from elbow plot we can choose 3 or 4 i'll go with 4 cluster.
#kmeans2=KMeans(n_clusters=2)
#kmeans3=KMeans(n_clusters=3)
kmeans=KMeans(n_clusters=2)
clusters=kmeans.fit_predict(df)
df["label"]=clusters

In [ ]:
#%% plot with cluster / center(centroid)
plt.figure(figsize=(20,8))
plt.scatter(df.DONOR_AGE [df.label==0],df.IN_HOUSE[df.label==0],color="red",alpha= 0.8)
plt.scatter(df.DONOR_AGE [df.label==1],df.IN_HOUSE[df.label==1],color="green",alpha= 0.8)
plt.scatter(kmeans.cluster_centers_[:,0],kmeans.cluster_centers_[:,1],color="orange")# cluster center(centroid)
plt.title("Kmeans(k=4) Cluster")
plt.xlabel("DONOR_AGE ")
plt.ylabel("IN_HOUSE")
plt.legend(df,loc='upper right')
plt.show()

In [ ]:
#%%   from elbow plot we can choose 3 or 4 i'll go with 4 cluster.
#kmeans2=KMeans(n_clusters=2)
#kmeans3=KMeans(n_clusters=3)
kmeans=KMeans(n_clusters=5)
clusters=kmeans.fit_predict(df)
df["label"]=clusters

In [ ]:
#%% plot with cluster / center(centroid)
plt.figure(figsize=(20,8))
plt.scatter(df.CLUSTER_CODE[df.label==0],df.CONTROL_NUMBER[df.label==0],color="red",alpha= 0.8)
plt.scatter(df.CLUSTER_CODE[df.label==1],df.CONTROL_NUMBER[df.label==1],color="green",alpha= 0.8)
plt.scatter(df.CLUSTER_CODE[df.label==2],df.CONTROL_NUMBER[df.label==2],color="blue",alpha= 0.8)
plt.scatter(df.CLUSTER_CODE[df.label==3],df.CONTROL_NUMBER[df.label==3],color="black",alpha= 0.8)
plt.scatter(df.CLUSTER_CODE[df.label==3],df.CONTROL_NUMBER[df.label==3],color="cyan",alpha= 0.8)
plt.scatter(kmeans.cluster_centers_[:,0],kmeans.cluster_centers_[:,1],color="orange")# cluster center(centroid)
plt.title("Kmeans(k=4) Cluster")
plt.xlabel("CLUSTER_CODE")
plt.ylabel("CONTROL_NUMBER")
plt.legend(df,loc='upper right')
plt.show()